We got this python notebook from the following GitHub repo (https://github.com/subpath/Keras_music_gereration) as a way to test our baseline:

### Music gereration firth Keras and TensorFlow backend

Plan was simple:
1. Read midi file, convert it to matrix of features
2. Create simple model with Keras and LSTM to learn the pattern
3. Use subsample of initial midi file as a input for model to generate pure art
4. Save prediction from model to midi file
.
.
.
5. PROFIT

<i> For disclamer: I've been using my old Dell Laptop with no GPU support</i>

In [2]:
import mido
import os
from mido import MidiFile, MidiTrack, Message
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
import numpy as np

Using TensorFlow backend.
/Users/StephanieBrito/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


##### Read midi file

In [3]:
#mid = MidiFile('SalsaGuantanamera.mid')  
midis = []
print(os.listdir('Data'))
for file in os.listdir('Data'):
    if file.endswith(".mid"):
        print(file)
        midis.append(MidiFile('Data/' + file, clip=True))
    #files.append(file)

['.DS_Store', 'abusowillieCol.mid', 'asifue_versionsalsa.mid', 'Bamboleo.mid', 'devorame-lalo.mid', 'lloraras.mid', 'MITIERRA[1].mid', 'mujerdearenaodeloeon.mid', 'pedronavaja-ruben.mid', 'quemaneraAlbita.mid', 'rumberaWchirino.mid', 'SalsaGuantanamera.mid']
abusowillieCol.mid
asifue_versionsalsa.mid
Bamboleo.mid
devorame-lalo.mid
lloraras.mid
MITIERRA[1].mid
mujerdearenaodeloeon.mid
pedronavaja-ruben.mid
quemaneraAlbita.mid
rumberaWchirino.mid
SalsaGuantanamera.mid


In [4]:
# mid = MidiFile('Data/abusowillieCol.mid', clip=True)

# for i, track in enumerate(mid.tracks):
#     print('Track {}: {}'.format(i, track.name))
#     for msg in track:
#         if msg.type == 'program_change':
#             print(msg)

##### Extract notes sequence

In [23]:
notes = []

def isPercussion(channel):
    return channel in range(8, 17) or channel in range(35, 82)

for file in os.listdir('Data')[1:6]:
    mid = MidiFile('Data/' + file, clip=True)
    for msg in mid:
        if msg.type == 'program_change':
            print(msg)
        if not msg.is_meta and (isPercussion(msg.channel)) and msg.type == 'note_on':
            data = msg.bytes()
            notes.append(data[1])

program_change channel=0 program=57 time=0.0026595666666666667
program_change channel=1 program=57 time=0
program_change channel=2 program=32 time=0
program_change channel=3 program=0 time=0
program_change channel=4 program=57 time=0
program_change channel=5 program=61 time=0
program_change channel=9 program=0 time=0
program_change channel=1 program=35 time=0
program_change channel=2 program=0 time=0
program_change channel=5 program=66 time=0
program_change channel=9 program=0 time=0
program_change channel=7 program=98 time=0
program_change channel=6 program=48 time=0
program_change channel=4 program=57 time=0
program_change channel=3 program=56 time=0
program_change channel=9 program=2 time=0
program_change channel=5 program=0 time=0
program_change channel=3 program=32 time=0
program_change channel=9 program=0 time=0
program_change channel=4 program=62 time=0
program_change channel=0 program=57 time=0
program_change channel=9 program=2 time=0
program_change channel=1 program=53 time=0

#### Apply min-max scalling

In [24]:
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(np.array(notes).reshape(-1,1))
notes = list(scaler.transform(np.array(notes).reshape(-1,1)))

/Users/StephanieBrito/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


#### Prepare features for training and data subsample for prediction

In [25]:
# LSTM layers requires that data must have a certain shape
# create list of lists fist
notes = [list(note) for note in notes]

# subsample data for training and prediction
X = []
y = []
# number of notes in a batch
n_prev = 80
for i in range(len(notes)-n_prev):
    X.append(notes[i:i+n_prev])
    y.append(notes[i+n_prev])
# save a seed to do prediction later
X_test = X[-300:]
X = X[:-300]
y = y[:-300]

#### Made sequential model with several layers, use LSTM as it time dependent data

I also whant to save checkpoints

In [26]:
model = Sequential()
model.add(LSTM(256, input_shape=(n_prev, 1), return_sequences=True))
model.add(LSTM(128, input_shape=(n_prev, 1), return_sequences=True))
model.add(LSTM(64, input_shape=(n_prev, 1), return_sequences=False))
model.add(Dense(1))
model.add(Activation('linear'))
optimizer = Adam(lr=0.001)
model.compile(loss='mse', optimizer=optimizer)
filepath="./Checkpoints/checkpoint_model_{epoch:02d}.hdf5"
model_save_callback = ModelCheckpoint(filepath, monitor='val_acc', 
                                      verbose=1, save_best_only=False, 
                                      mode='auto', period=5)

#### Train your model.
It might take a while, I was waiting for 1 hour with just 5 epoch

In [ ]:
ll = model.fit(np.array(X), np.array(y), 32, 5, verbose=1)

Epoch 1/5
43082/43082 [==============================] - 599s 14ms/step - loss: 0.0467
Epoch 2/5
43082/43082 [==============================] - 582s 14ms/step - loss: 0.0418
Epoch 3/5
 1248/43082 [..............................] - ETA: 9:23 - loss: 0.0390

#### Make a prediction

In [22]:
prediction = model.predict(np.array(X_test))
prediction = np.squeeze(prediction)
prediction = np.squeeze(scaler.inverse_transform(prediction.reshape(-1,1)))
prediction = [int(i) for i in prediction]
print(X_test)

[[[0.953125], [0.921875], [0.109375], [0.921875], [0.765625], [0.953125], [0.578125], [0.578125], [0.953125], [0.765625], [0.921875], [0.921875], [0.109375], [0.8125], [0.953125], [0.765625], [0.578125], [0.8125], [0.078125], [0.078125], [0.828125], [0.640625], [0.875], [0.6875], [0.078125], [0.828125], [0.640625], [0.875], [0.6875], [0.578125], [0.765625], [0.953125], [0.921875], [0.078125], [0.921875], [0.765625], [0.953125], [0.578125], [0.578125], [0.953125], [0.765625], [0.046875], [0.046875], [0.640625], [0.828125], [0.953125], [0.578125], [0.875], [0.765625], [0.015625], [0.640625], [0.828125], [0.875], [0.5625], [0.75], [0.9375], [0.953125], [0.921875], [0.015625], [0.921875], [0.953125], [0.0], [0.625], [0.8125], [0.0], [0.9375], [0.5625], [0.671875], [0.75], [0.859375], [0.8125], [0.625], [0.671875], [0.859375], [0.578125], [0.765625], [0.953125], [0.109375], [0.921875], [0.921875]], [[0.921875], [0.109375], [0.921875], [0.765625], [0.953125], [0.578125], [0.578125], [0.95312

#### Save your result to new midi file

In [12]:
mid = MidiFile()
track = MidiTrack()
t = 0
for note in prediction:
    # 147 means note_on
    # 67 is velosity
    note = np.asarray([147, note, 67])
    bytes = note.astype(int)
    msg = Message.from_bytes(bytes[0:3])
    t += 1
    msg.time = t
    track.append(msg)
mid.tracks.append(track)
mid.save('5_epoch_piano.mid')

#### Just listen to it. The result is surreal!